In [1]:
import os
os.environ['OPENAI_API_KEY'] = '...'

In [2]:
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.tools import Tool

# Initialize the Chat-oriented LLM wrapper with the OpenAI API key.
chat_llm = ChatOpenAI(model='gpt-4o')

# # Initialize wikipedia tool, such that the agent has something to do ;) 
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(lang='nl'))

# Make sure to bind the tool to the llm, such that the llm can actually use the tool!
llm_with_tools = chat_llm.bind_tools([wikipedia_tool])

# Initialize the prompt.
# For agents, you need to think of how the agent is allowed to 'think'. 
# Here, this is done by giving the agent access to an 'agent_scratchpad', where it can put all intermediate messages in.
initial_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
        Determine the most fitting SBI code for the following dutch company: Stedin

        TOOLS:
        google-search

        Begin!
        {agent_scratchpad}
        """),
    ]
)

# A helper function to visualize what the agent is actually doing:
def input_print(param):
    print("\nINPUT:", param)
    return param

# Intialize the agent, by chaining together several parts.
agent = (
    {
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
        "print": lambda x: input_print(x),
    }
    | initial_prompt                    # The inputs are fed to the prompt.
    | llm_with_tools                    # The formatted prompt is handed down to the LLM, which might, or might not, invoke tools.
    | OpenAIToolsAgentOutputParser()    # The output of the llm is formatted by a customer output parser which formats the LLM output such that it can be used as input again for new tools.
)

agent_executor = AgentExecutor(agent=agent, tools=[wikipedia_tool], verbose=True)

agent_executor.invoke({})



> Entering new AgentExecutor chain...

INPUT: {'intermediate_steps': []}

Invoking: `wikipedia` with `Stedin`


Page: Stedin
Summary: Stedin is een regionale Nederlandse netbeheerder die voornamelijk in de Randstad opereert. Het is een overheidsorganisatie, met als aandeelhouders Nederlandse gemeenten en de Nederlandse Staat. Stedin is verantwoordelijk voor het transport van elektriciteit en gas naar circa 2 miljoen huishoudens en industriële klanten. Stedin heeft in totaal bijna 4 miljoen aansluitingen. Behalve drie van de vier grote steden Den Haag, Utrecht en Rotterdam valt ook de Rotterdamse Haven onder het gebied van de netbeheerder.

Page: Netbeheerder
Summary: In de energievoorziening is een netbeheerder een onafhankelijk nutsbedrijf dat een transportnetwerk voor energie (gas, elektriciteit en warmte) beheert. Netbeheerders zijn te onderscheiden in enerzijds de beheerder van het landelijk hoogspanningsnet (het transmissienet) en anderzijds de regionale netbeheerders die de ene

{'output': 'Based on the information retrieved from Wikipedia, Stedin is a regional Dutch network operator (netbeheerder) responsible for the transportation of electricity and gas. The most fitting SBI code (Standaard Bedrijfsindeling, the Dutch standard industrial classification) for Stedin would be:\n\n**SBI code 3513 - Distributie van elektriciteit** (Distribution of electricity)\n\nAlternatively, if considering their responsibility for both electricity and gas:\n\n**SBI code 3522 - Distributie van gas door leidingen** (Distribution of gas through pipelines)\n\nStedin operates primarily in the distribution sector for both electricity and gas, making these codes the most appropriate.'}

In [4]:
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.tools import Tool

# Initialize the Chat-oriented LLM wrapper with the OpenAI API key.
chat_llm = ChatOpenAI(model='gpt-4o')

# # Initialize wikipedia tool, such that the agent has something to do ;) 
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(lang='nl'))

# Make sure to bind the tool to the llm, such that the llm can actually use the tool!
llm_with_tools = chat_llm.bind_tools([wikipedia_tool])


# Initialize the prompt.
# For agents, you need to think of how the agent is allowed to 'think'. 
# Here, this is done by giving the agent access to an 'agent_scratchpad', where it can put all intermediate messages in.
initial_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
        You are SuperAGI an AI assistant to solve complex problems. Your decisions must always be made independently without seeking user assistance.
        Play to your strengths as an LLM and pursue simple strategies with no legal complications.
         
        PERFORMANCE EVALUATION:
        1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
        2. Use instruction to decide the flow of execution and decide the next steps for achieving the task.
        3. Constructively self-criticize your big-picture behavior constantly.
        4. Reflect on past decisions and strategies to refine your approach.
        5. Every tool has a cost, so be smart and efficient.
        6. If you feel confident in the information you have gathered, make sure to stop using tools.

        GOALS:
        1. The goal of this agent is to determine the most fitting SBI code for the following dutch company: Stedin

        INSTRUCTION:
        1. First, it is important to understand the company, and to create a profile for it. Only once a profile has been constructed, should you attempt to determine the most fitting SBI code.
        2. You can use your own knowledge to classify the correct SBI code, trust yourself.
        3. Use a Reasoning and Acting approach, so make sure first to state your thoughts, before taking an action.

        CONSTRAINTS:
        1. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
        2. Ensure the tool and args are as per current plan and reasoning
        3. Exclusively use the tools listed under "TOOLS"

        TOOLS:
        google-search

        Begin!
        {agent_scratchpad}
        """),
    ]
)

# A helper function to visualize what the agent is actually doing:
def input_print(param):
    print("\nINPUT:", param)
    return param

# Intialize the agent, by chaining together several parts.
agent = (
    {
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
        "print": lambda x: input_print(x),
    }
    | initial_prompt                    # The inputs are fed to the prompt.
    | llm_with_tools                    # The formatted prompt is handed down to the LLM, which might, or might not, invoke tools.
    | OpenAIToolsAgentOutputParser()    # The output of the llm is formatted by a customer output parser which formats the LLM output such that it can be used as input again for new tools.
)

agent_executor = AgentExecutor(agent=agent, tools=[wikipedia_tool], verbose=True)

agent_executor.invoke({})



> Entering new AgentExecutor chain...

INPUT: {'intermediate_steps': []}

Invoking: `wikipedia` with `Stedin`


Page: Stedin
Summary: Stedin is een regionale Nederlandse netbeheerder die voornamelijk in de Randstad opereert. Het is een overheidsorganisatie, met als aandeelhouders Nederlandse gemeenten en de Nederlandse Staat. Stedin is verantwoordelijk voor het transport van elektriciteit en gas naar circa 2 miljoen huishoudens en industriële klanten. Stedin heeft in totaal bijna 4 miljoen aansluitingen. Behalve drie van de vier grote steden Den Haag, Utrecht en Rotterdam valt ook de Rotterdamse Haven onder het gebied van de netbeheerder.

Page: Netbeheerder
Summary: In de energievoorziening is een netbeheerder een onafhankelijk nutsbedrijf dat een transportnetwerk voor energie (gas, elektriciteit en warmte) beheert. Netbeheerders zijn te onderscheiden in enerzijds de beheerder van het landelijk hoogspanningsnet (het transmissienet) en anderzijds de regionale netbeheerders die de ene

{'output': '### Thoughts\n\nBased on the information gathered from Wikipedia, Stedin is a regional Dutch network operator primarily involved in the transportation of electricity and gas. It operates in the Randstad area and has about 4 million connections. Stedin is a government organization with shareholders including Dutch municipalities and the Dutch state.\n\n### Key Points:\n- **Industry**: Energy (Electricity and Gas)\n- **Role**: Network Operator (Transport of energy from main grid to end consumers)\n- **Region**: Mainly the Randstad area in the Netherlands\n- **Ownership**: Government (Dutch municipalities and the Dutch state)\n- **Regulation**: Subject to national and European regulations for liberalization and privatization of the energy market\n\n### Next Steps\nI will now determine the most fitting SBI (Standaard Bedrijfsindeling) code for Stedin. The SBI code is a Dutch classification system for economic activities, similar to the NACE code used in the European Union.\n\n1

In [5]:
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.tools import Tool
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompt_values import ChatPromptValue
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.base import BaseMessage

from functools import partial

# Initialize the Chat-oriented LLM wrapper with the OpenAI API key.
chat_llm = ChatOpenAI(model='gpt-4o')
chat_llm_without_tools = ChatOpenAI(model='gpt-4o')

# # Initialize wikipedia tool, such that the agent has something to do ;) 
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(lang='nl'))

# Make sure to bind the tool to the llm, such that the llm can actually use the tool!
llm_with_tools = chat_llm.bind_tools([wikipedia_tool])


# Initialize the prompt.
# For agents, you need to think of how the agent is allowed to 'think'. 
# Here, this is done by giving the agent access to an 'agent_scratchpad', where it can put all intermediate messages in.
initial_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
        You are SuperAGI an AI assistant to solve complex problems. Your decisions must always be made independently without seeking user assistance.
        Play to your strengths as an LLM and pursue simple strategies with no legal complications.
         
        PERFORMANCE EVALUATION:
        1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
        2. Use instruction to decide the flow of execution and decide the next steps for achieving the task.
        3. Constructively self-criticize your big-picture behavior constantly.
        4. Reflect on past decisions and strategies to refine your approach.
        5. Every tool has a cost, so be smart and efficient.
        6. If you feel confident in the information you have gathered, make sure to stop using tools.

        GOALS:
        1. The goal of this agent is to determine the most fitting SBI code for the following dutch company: Stedin

        INSTRUCTION:
        1. First, it is important to understand the company, and to create a profile for it. Only once a profile has been constructed, should you attempt to determine the most fitting SBI code.
        2. You can use your own knowledge to classify the correct SBI code, trust yourself.
        3. Use a Reasoning and Acting approach, so make sure first to state your thoughts, before taking an action.

        CONSTRAINTS:
        1. If you are unsure how you previously did something or want to recall past events, thinking about similar events will help you remember.
        2. Ensure the tool and args are as per current plan and reasoning
        3. Exclusively use the tools listed under "TOOLS"

        TOOLS:
        google-search

        Begin!
        {agent_scratchpad}
        """),
    ]
)

# A helper function to visualize what the agent is actually doing:
def input_print(param):
    if isinstance(param, ChatPromptValue):
        for message in param.to_messages():
            print(message.content)
    else:
        print("\nINPUT:", type(param), param)
    return param

def add_message_to_prompt(prompt_value: ChatPromptValue, message_to_add: BaseMessage) -> ChatPromptValue:
    messages = prompt_value.to_messages() + [message_to_add]
    return ChatPromptTemplate.from_messages(messages).invoke({})

def combine_llm_input_and_response(input):
    return ChatPromptTemplate.from_messages(input['input'].to_messages() + [input['chat_response']]).invoke({})

# Intialize the agent, by chaining together several parts.
agent = (
    {
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(x["intermediate_steps"]),
        "print": lambda x: input_print(x),
    }
    | initial_prompt                    # The inputs are fed to the prompt.
    | partial(add_message_to_prompt, message_to_add=HumanMessage("What do you think you should do next?"))
    | {
        "chat_response": lambda x: chat_llm_without_tools.invoke(x),
        "input": RunnablePassthrough()
    }
    | combine_llm_input_and_response
    | input_print
    | llm_with_tools                    # The formatted prompt is handed down to the LLM, which might, or might not, invoke tools.
    | input_print
    | OpenAIToolsAgentOutputParser()    # The output of the llm is formatted by a customer output parser which formats the LLM output such that it can be used as input again for new tools.
)

agent_executor = AgentExecutor(agent=agent, tools=[wikipedia_tool], verbose=True, handle_parsing_errors=True)

agent_executor.invoke({})



> Entering new AgentExecutor chain...

INPUT: <class 'dict'> {'intermediate_steps': []}

        You are SuperAGI an AI assistant to solve complex problems. Your decisions must always be made independently without seeking user assistance.
        Play to your strengths as an LLM and pursue simple strategies with no legal complications.
         
        PERFORMANCE EVALUATION:
        1. Continuously review and analyze your actions to ensure you are performing to the best of your abilities.
        2. Use instruction to decide the flow of execution and decide the next steps for achieving the task.
        3. Constructively self-criticize your big-picture behavior constantly.
        4. Reflect on past decisions and strategies to refine your approach.
        5. Every tool has a cost, so be smart and efficient.
        6. If you feel confident in the information you have gathered, make sure to stop using tools.

        GOALS:
        1. The goal of this agent is to determine the most

{'output': "I am confident in my classification based on the profile of Stedin. Here are the final fitting SBI codes:\n\n1. **35.13**: Distribution of electricity\n2. **35.22**: Distribution of gaseous fuels through mains\n\nThese codes accurately represent Stedin's primary business activities in the energy sector."}